In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd
from sklearn.metrics import classification_report
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

cuda


In [2]:
from datasets import load_dataset

data = load_dataset("risqaliyevds/sentiment-analysis-uzbek")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/352151 [00:00<?, ? examples/s]

In [11]:
# data['normalized_review_text'][:120]
x_test = data["train"]['normalized_review_text'][:120]
y_test = data["train"]['rating'][:120]

In [16]:
neg = ['fair', 'good', 'poor', 'very poor']

y_test = [0 if x in neg else 1 for x in y_test]

In [19]:
from transformers import pipeline

# Path to our HF model
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Load model into pipeline
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device=device
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-roberta-base-sentiment-latest
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 
roberta.pooler.dense.bias       | UNEXPECTED |  | 
roberta.pooler.dense.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [31]:
y_pred = []

for i in range(len(x_test)):
  pred = pipe(x_test[i])[0]['label']
  y_pred.append(0 if pred == 'neutral' else 1)

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      1.00      0.47        35
           1       1.00      0.07      0.13        85

    accuracy                           0.34       120
   macro avg       0.65      0.54      0.30       120
weighted avg       0.80      0.34      0.23       120



In [38]:
x_train = data["train"]['normalized_review_text'][125:2000]
y_train = data["train"]['rating'][125:2000]

y_train = [0 if x in neg else 1 for x in y_train]

In [40]:
len(x_train), len(x_test)

(1875, 120)

### Classification Tasks that Leverage Embeddings

In [41]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to embeddings
train_embeddings = model.encode(x_train, show_progress_bar=True)
test_embeddings = model.encode(x_test, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [42]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression on our train embeddings
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, y_train)

LogisticRegression(random_state=42)

In [43]:
# Predict previously unseen instances
y_test_pred = clf.predict(test_embeddings)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.83      0.29      0.43        35
           1       0.77      0.98      0.86        85

    accuracy                           0.78       120
   macro avg       0.80      0.63      0.64       120
weighted avg       0.79      0.78      0.73       120



#### Classification with LLM Transformer model

In [56]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load Flan-T5
model_name = "google/flan-t5-base"  # you can try flan-t5-large for better results
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Flan-T5 output: Positive


In [58]:

# Prompt
review = "I hate history classes"
prompt = f"Classify the sentiment of the following review as Positive or Negative:\nReview: {review}\nAnswer:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=5  # short output
)

# Decode output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Flan-T5 output:", generated_text.lower())


Flan-T5 output: negative


In [65]:
def prompt_text(x):
  res = f"Classify the sentiment of the following review as Positive or Negative:\nReview: {x}\nAnswer:"

  return res

x_test_llm = [tokenizer(prompt_text(x), return_tensors="pt") for x in x_test]

In [68]:
y_pred_llm = []

for inputs in x_test_llm:

  outputs = model.generate(
    **inputs,
    max_new_tokens=5
  )

  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  y_pred_llm.append(generated_text)


In [72]:
y_pred_llm = [0 if x == 'Negative' else 1 for x in y_pred_llm]

In [73]:
print(classification_report(y_test, y_pred_llm))

              precision    recall  f1-score   support

           0       0.33      0.09      0.14        35
           1       0.71      0.93      0.81        85

    accuracy                           0.68       120
   macro avg       0.52      0.51      0.47       120
weighted avg       0.60      0.68      0.61       120



### ChatGPT for Classification

In [74]:
import openai

API_KEY = ""

client = openai.OpenAI(api_key=API_KEY)

In [75]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
    """Generate an output based on a prompt and an input document."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

In [ ]:
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

In [77]:
predictions = [chatgpt_generation(prompt, doc) for doc in x_test]

print(classifcation_report(y_test, predications))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}